# Cvičení: Gold pohledy

## Krok 1: Import potřebných knihoven.

In [ ]:
# TODO

## Krok 2: Nastavte prostředí a katalogy.

In [ ]:
# TODO

## Krok 3: Napište pomocnou funkci `run_gold_view_queries`.

In [ ]:
# TODO

## Krok 4: Připravte SQL dotazy pro vytvoření gold views.

In [ ]:
# TODO

## Krok 5: Spusťte funkci pro vytvoření všech views.

In [ ]:
# TODO